In [1]:
from tinydb import TinyDB, Query
import os
import json
import numpy as np

class DB:
    def __init__(self, db_name: str) -> None:
        self.db_instance = TinyDB(db_name)
        self.tables = {field: self.db_instance.table(field) for field in ["reality", "objects", "rules"]}
        
        # Defaults
        self.OBJECTS = {
        "name": None,
        "id": None,
        "cat": "object",
        "desc": "An object.",
        "initRequire": [],
        "constRequire": [],
        "initGains": [],
        "constGains": [],
        "replace": [],
        "restrict": [],
        "visible": True
        }
        self.RULES = {
            "dismantleGain": 0.5
        }
        self.REALITY = {
            "id": None,
            "type": None,
            "amount": None
        }

class Compiler(DB):
    def __init__(self, db_name: str, scenario_dir: str) -> None:
        super().__init__(db_name)
        self.scenario_dir = scenario_dir

    def fill_empty(self, obj: dict, default_vals: dict) -> dict:
        for key in default_vals.keys():
            if key not in obj.keys():
                obj.update({key: default_vals[key]})
        return obj


    def integrate_objects(self, content: list[dict]) -> list[dict]:
        error_handle = []
        for element in content: 
            if len([el for el in element.keys() if el in ["name", "id"]]) == 2:
                content[content.index(element)] = self.fill_empty(element, self.OBJECTS)
            else:
                error_handle.append(json.dumps(element))

        if not error_handle:
            return content
        else:
            err = '\n'.join(error_handle)
            print(f"No mandatory fields id and name in objects: indexes: {err}")
                    

    def compile_scenario(self) -> None:
        valid_scenario_files = [f for f in os.listdir(self.scenario_dir) if f.endswith(".json")]
        
        for file in valid_scenario_files:
            with open(f"{self.scenario_dir}/{file}", "r") as f:
                content = json.loads(f.read())
                for key in content.keys():
                    key = key.lower()
                    
                    if key == "objects":
                        integrated_content = self.integrate_objects(content[key])
                        self.tables[key].insert_multiple(integrated_content)
                    elif key == "rules":
                        self.tables[key].insert(self.fill_empty(content[key], self.RULES))
                    elif key == "reality":
                        for real in content[key]:
                            if not np.isin(list(real.keys()), list(self.REALITY.keys())).all():
                                print("Some reality elements does not fit schema.")
                                exit()
                        self.tables[key].insert_multiple(content[key])


In [7]:
class Queue(DB):
    def __init__(self, db_name: str, queue: list[dict]) -> None:
        super().__init__(db_name)
        self.queue = queue
        self.agg = {"into_reality": [], "into_void": [], "cost": [], "gain": [], "err": []} 
        self.summary = {}

    def objects_agg(self) -> None:
        objects = [obj for obj in self.queue if obj["targetType"] == "objects"]
        search = self.tables["objects"].search(Query()["id"].one_of([i["targetId"] for i in objects]))

        if "intoVoid" in [x["action"] for x in objects]:
            dismantle_rate = self.tables["rules"].all()[0]["dismantleGain"]

        for obj in objects:
            obj_props = [s for s in search if s["id"] == obj["targetId"]][0]

            if obj["action"] == "intoReality":
                self.agg["into_reality"].append({"id": obj_props["id"], "type": "objects"})
                [self.agg["cost"].append(c) for c in obj_props["initRequire"]]
                [self.agg["gain"].append(g) for g in obj_props["initGains"]]

            if obj["action"] == "intoVoid":
                self.agg["into_void"].append({"id": obj_props["id"], "type": "objects"})
                for dsmntl in obj_props["initRequire"]:
                    newVal = dsmntl["amount"] * dismantle_rate
                    self.agg["gain"].append({"id": dsmntl["id"], "type": "objects", "amount": newVal})
                
                    

    def queue_summary(self):
        for mode in ["into_reality", "into_void"]: 
            unique, counts = np.unique(self.agg[mode], return_counts=True)
            self.summary.update({mode: [(unique[i], counts[i]) for i in range(0, len(unique))]})

        for mode in ["cost", "gains"]:
            pass
        
        

    def compile_queue(self):
        self.objects_agg()
        #self.queue_summary()

In [8]:
q = [
    {"action": "intoReality", "targetId": "hut_wood", "targetType": "objects"},
    {"action": "intoReality", "targetId": "hut_wood", "targetType": "objects"}
]
que = Queue("db.json", q)

In [9]:
que.compile_queue()

In [11]:
que.queue_summary()

TypeError: '<' not supported between instances of 'dict' and 'dict'

In [283]:
unique, counts = np.unique([], return_counts=True)p

In [284]:
print(unique, counts)

[] []


In [167]:
Compiler("scenario", "db.json").compile_scenario()

IsADirectoryError: [Errno 21] Is a directory: 'scenario'

In [36]:
tt = Query()
x.tables["reality"].insert({"name": "Mikolaj"})

1

In [5]:
x.tables["objects"].search(tt.id == "hut_wood")

NameError: name 'x' is not defined

In [39]:
x.tables["reality"].remove(tt.name == "Mikolaj")

[1]

In [40]:
x.tables["reality"].search(tt.name == "Mikolaj")

[]

In [42]:
x.tables.keys()

dict_keys(['reality', 'concepts', 'rules'])

In [62]:
[1,2, 3] == [1,2]

False

In [1]:
%pip install numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 25.9 MB/s eta 0:00:00m eta 0:00:010:01:01
Note: you may need to restart the kernel to use updated packages.
